In [ ]:
#!pip install schedule

# Imported Libraries

In [1]:
import pandas as pd
import numpy as np
import sqlite3
import shutil
from datetime import datetime
import os
import schedule
import time
import warnings
warnings.filterwarnings("ignore")

# Loading Dataset

In [2]:
df=pd.read_csv("FashionDataset.csv")

# Task A – Data Preparation and Database Integration

In [3]:
df['MRP'] = df['MRP'].astype(str)
df['SellPrice'] = df['SellPrice'].astype(str)
df['MRP'] = df['MRP'].str.replace('Rs\n', '').astype(float)
df['SellPrice'] = df['SellPrice'].str.replace('Rs\n', '').astype(float)

In [4]:
df['Sizes'] = df['Sizes'].str.replace('Size:', '')
df = df.assign(Sizes=df['Sizes'].str.split(',')).explode('Sizes')

In [5]:
df = df.rename(columns={'Deatils': 'Details'})

In [6]:
df['Discount'] = df['Discount'].astype(str)
df['Discount'] = df['Discount'].str.replace('% off', '').astype(float)

In [8]:
df.rename(columns={'Unnamed: 0': 'ID'}, inplace=True)

In [9]:
df

,ID,BrandName,Details,Sizes,MRP,SellPrice,Discount,Category
0,0,life,solid cotton blend collar neck womens a-line d...,Large,1699.0,849.0,50.0,Westernwear-Women
1,0,life,solid cotton blend collar neck womens a-line d...,Medium,1699.0,849.0,50.0,Westernwear-Women
2,0,life,solid cotton blend collar neck womens a-line d...,Small,1699.0,849.0,50.0,Westernwear-Women
3,0,life,solid cotton blend collar neck womens a-line d...,X-Large,1699.0,849.0,50.0,Westernwear-Women
4,0,life,solid cotton blend collar neck womens a-line d...,X-Small,1699.0,849.0,50.0,Westernwear-Women
...,...,...,...,...,...,...,...,...
58418,8,ayesha,womens metallic silver double drop with black ...,Nan,298.0,149.0,50.0,Jewellery-Women
58419,14,jewelz,beautiful white crystal silver plated earrings,Nan,1739.0,608.0,65.0,Jewellery-Women
58420,15,ayesha,womens metallic gold statement multilayered ch...,Nan,798.0,399.0,50.0,Jewellery-Women
58421,16,jewelz,nine pieces combo fashion non-precious metal b...,Nan,1339.0,468.0,65.0,Jewellery-Women


In [10]:
cleaned_df = df.dropna()
cleaned_df  = cleaned_df .drop(columns=['ID'])

In [11]:
cleaned_df

,BrandName,Details,Sizes,MRP,SellPrice,Discount,Category
0,life,solid cotton blend collar neck womens a-line d...,Large,1699.0,849.0,50.0,Westernwear-Women
1,life,solid cotton blend collar neck womens a-line d...,Medium,1699.0,849.0,50.0,Westernwear-Women
2,life,solid cotton blend collar neck womens a-line d...,Small,1699.0,849.0,50.0,Westernwear-Women
3,life,solid cotton blend collar neck womens a-line d...,X-Large,1699.0,849.0,50.0,Westernwear-Women
4,life,solid cotton blend collar neck womens a-line d...,X-Small,1699.0,849.0,50.0,Westernwear-Women
...,...,...,...,...,...,...,...
58418,ayesha,womens metallic silver double drop with black ...,Nan,298.0,149.0,50.0,Jewellery-Women
58419,jewelz,beautiful white crystal silver plated earrings,Nan,1739.0,608.0,65.0,Jewellery-Women
58420,ayesha,womens metallic gold statement multilayered ch...,Nan,798.0,399.0,50.0,Jewellery-Women
58421,jewelz,nine pieces combo fashion non-precious metal b...,Nan,1339.0,468.0,65.0,Jewellery-Women


In [24]:
cleaned_df.head()

,BrandName,Details,Sizes,MRP,SellPrice,Discount,Category
0,life,solid cotton blend collar neck womens a-line d...,Large,1699.0,849.0,50.0,Westernwear-Women
1,life,solid cotton blend collar neck womens a-line d...,Medium,1699.0,849.0,50.0,Westernwear-Women
2,life,solid cotton blend collar neck womens a-line d...,Small,1699.0,849.0,50.0,Westernwear-Women
3,life,solid cotton blend collar neck womens a-line d...,X-Large,1699.0,849.0,50.0,Westernwear-Women
4,life,solid cotton blend collar neck womens a-line d...,X-Small,1699.0,849.0,50.0,Westernwear-Women


In [12]:
filtered_df = cleaned_df[cleaned_df['Discount'] > 30]
filtered_df.head()

,BrandName,Details,Sizes,MRP,SellPrice,Discount,Category
0,life,solid cotton blend collar neck womens a-line d...,Large,1699.0,849.0,50.0,Westernwear-Women
1,life,solid cotton blend collar neck womens a-line d...,Medium,1699.0,849.0,50.0,Westernwear-Women
2,life,solid cotton blend collar neck womens a-line d...,Small,1699.0,849.0,50.0,Westernwear-Women
3,life,solid cotton blend collar neck womens a-line d...,X-Large,1699.0,849.0,50.0,Westernwear-Women
4,life,solid cotton blend collar neck womens a-line d...,X-Small,1699.0,849.0,50.0,Westernwear-Women


# Establishing a connection with SQL

In [13]:
# Establish a connection to an SQLite database
conn = sqlite3.connect('fashionretail.db')
cursor = conn.cursor()

# Drop the table if it already exists
cursor.execute('DROP TABLE IF EXISTS fashion_sales')
conn.commit()

In [14]:
# Create a table in the database
cursor.execute('''
CREATE TABLE IF NOT EXISTS fashion_sales (
    id INTEGER PRIMARY KEY,
    BrandName TEXT,
    Details TEXT,
    Sizes TEXT,
    MRP REAL,
    SellPrice REAL,
    Discount REAL,
    Category TEXT
)
''')
conn.commit()

# Import the dataset into a table within the database

In [15]:
# Insert the dataset into the table
filtered_df .to_sql('fashion_sales', conn, if_exists='append', index=False)

58287

# Retrieve and display records or documents from the table

In [16]:
# Retrieve and display records from the table
query = "SELECT * FROM fashion_sales LIMIT 5"
results = pd.read_sql_query(query, conn)
print(results)

   id BrandName                                            Details    Sizes  \
0   1      life  solid cotton blend collar neck womens a-line d...    Large   
1   2      life  solid cotton blend collar neck womens a-line d...   Medium   
2   3      life  solid cotton blend collar neck womens a-line d...    Small   
3   4      life  solid cotton blend collar neck womens a-line d...  X-Large   
4   5      life  solid cotton blend collar neck womens a-line d...  X-Small   

      MRP  SellPrice  Discount           Category  
0  1699.0      849.0      50.0  Westernwear-Women  
1  1699.0      849.0      50.0  Westernwear-Women  
2  1699.0      849.0      50.0  Westernwear-Women  
3  1699.0      849.0      50.0  Westernwear-Women  
4  1699.0      849.0      50.0  Westernwear-Women  


# Sort the records or documents based on a given condition.

In [17]:
# Example: Sort records by SellPrice in descending order
sorted_query = "SELECT * FROM fashion_sales ORDER BY SellPrice DESC"
sorted_results = pd.read_sql_query(sorted_query, conn)
print(sorted_results.head())

      id       BrandName                                            Details  \
0  57592           coach  womens 33.25 mm preston mother of pearl dial s...   
1  57394  emporio armani  womens gianni t-bar mother of pearl dial analo...   
2  57574           coach  womens grey stainless steel analogue watch  - ...   
3  57343           coach  womens purple dial metallic analogue watch - c...   
4  57260           coach  womens park blue dial stainless steel analogue...   

  Sizes      MRP  SellPrice  Discount       Category  
0   Nan  28495.0    17097.0      40.0  Watches-Women  
1   Nan  24495.0    14697.0      40.0  Watches-Women  
2   Nan  22995.0    13797.0      40.0  Watches-Women  
3   Nan  22500.0    13500.0      40.0  Watches-Women  
4   Nan  22000.0    13200.0      40.0  Watches-Women  


# Count the number of records or documents present in the table

In [18]:
# Count the number of records in the table
count_query = "SELECT COUNT(*) FROM fashion_sales"
count_result = pd.read_sql_query(count_query, conn)
print(count_result)

   COUNT(*)
0     58287


# Perform grouping operations on records

In [19]:
# Example: Group by Category and calculate average SellPrice
group_query = "SELECT Category, AVG(SellPrice) as Avg_SellPrice FROM fashion_sales GROUP BY Category"
grouped_results = pd.read_sql_query(group_query, conn)
print(grouped_results)

                   Category  Avg_SellPrice
0            Footwear-Women    1388.238132
1           Fragrance-Women    1286.625000
2          Indianwear-Women    1101.899411
3           Jewellery-Women     799.131783
4  Lingerie&Nightwear-Women     597.411088
5             Watches-Women    4699.743243
6         Westernwear-Women     919.642775


# Execute update operations on records

In [20]:
# Example: Update Discount for a specific BrandName
update_query = "UPDATE fashion_sales SET Discount = 55 WHERE BrandName = 'life'"
cursor.execute(update_query)
conn.commit()

# Verify the update
verify_query = "SELECT * FROM fashion_sales WHERE BrandName = 'life'"
updated_results = pd.read_sql_query(verify_query, conn)
print(updated_results)

         id BrandName                                            Details  \
0         1      life  solid cotton blend collar neck womens a-line d...   
1         2      life  solid cotton blend collar neck womens a-line d...   
2         3      life  solid cotton blend collar neck womens a-line d...   
3         4      life  solid cotton blend collar neck womens a-line d...   
4         5      life  solid cotton blend collar neck womens a-line d...   
...     ...       ...                                                ...   
2708  51518      life             women cotton regular sleepwear - multi   
2709  51519      life             women cotton regular sleepwear - multi   
2710  51520      life             women cotton regular sleepwear - multi   
2711  51521      life             women cotton regular sleepwear - multi   
2712  51522      life             women cotton regular sleepwear - multi   

        Sizes     MRP  SellPrice  Discount                  Category  
0       Large  1

# Task B – Data Export, Migration and Backup

Export data

In [21]:
# Export data from the sales table to a CSV file
export_file_path = 'C:/Users/hp/Downloads/PythonWork/sales.csv'
filtered_df.to_csv(export_file_path, index=False)
print(f'Data exported to {export_file_path}')

Data exported to C:/Users/hp/Downloads/PythonWork/sales.csv


Backup

In [22]:
# Source and destination paths
source_file_path = 'C:/Users/hp/Downloads/PythonWork/sales.csv'
backup_dir = 'C:/Users/hp/Downloads/PythonWork/backup/'

# Create the backup directory if it doesn't exist
os.makedirs(backup_dir, exist_ok=True)

# Create a timestamped backup file name
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
backup_file_path = f'{backup_dir}sales_{timestamp}.csv'

# Copy the file
shutil.copy(source_file_path, backup_file_path)
print(f'Backup created at {backup_file_path}')

Backup created at C:/Users/hp/Downloads/PythonWork/backup/sales_20240803_121711.csv


Error Handling and Automated Backup

The Below Code will run indefinitely as the scheduling process is executing. It will check for backup in every 1 minute and that is why the keyboard interrupt is showing due to stop the execution

In [23]:
def backup_job():
    try:
        # Define the source directory and the backup directory
        source_dir = 'C:/Users/hp/Downloads/PythonWork/'
        backup_dir = 'C:/Users/hp/Downloads/PythonWork/backup/'
        
        # Create the backup directory if it doesn't exist
        os.makedirs(backup_dir, exist_ok=True)
        
        # Copy all files from the source directory to the backup directory
        for filename in os.listdir(source_dir):
            full_file_name = os.path.join(source_dir, filename)
            if os.path.isfile(full_file_name):
                shutil.copy(full_file_name, backup_dir)
        
        print(f"Backup completed successfully to {backup_dir}")
    except Exception as e:
        print(f"Backup failed: {e}")

# Schedule the backup job to run daily at a specified time
schedule.every(1).minutes.do(backup_job)

# Run the scheduled tasks
while True:
    schedule.run_pending()
    time.sleep(10)


Backup completed successfully to C:/Users/hp/Downloads/PythonWork/backup/


KeyboardInterrupt: 